## 实战案例：预测厄瓜多尔零售商的不同商店出售的数千种商品的单位销量

In [1]:
# 导入数据
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder 
import lightgbm as lgb 
from datetime import date, timedelta


In [2]:
# 根目录
path = 'D:/Program Projects/Python Projects/temp/厄瓜多尔超市销量预测/input/'

In [3]:


# unit_sales 进行 log1p() 预处理，好处是可以 对偏度比较大的数据进行转化，将其压缩到一个较小的区间，
# 最后 log1p() 预处理 能起到平滑数据的作用。另外在评价指标部分也是对 unit_sales 进行同样的处理，这部分操作也是预处理。

# 另一个操作是对 date 进行处理，将表格文件中的时间字符串转换成日期格式。提 前处理不仅有便于后续操作，还能减少代码量。
df_train = pd.read_csv(path+'train.csv', converters={'unit_sales':lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, parse_dates=["date"])

df_test = pd.read_csv(path + "test.csv",parse_dates=["date"]) 
items = pd.read_csv(path+'items.csv') 
stores = pd.read_csv(path+'stores.csv') 

# 类型转换 
df_train['onpromotion'] = df_train['onpromotion'].astype(bool) 
df_test['onpromotion'] = df_test['onpromotion'].astype(bool)

C:\Users\yx140\AppData\Local\Temp\ipykernel_24996\956254603.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(path+'train.csv', converters={'unit_sales':lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, parse_dates=["date"])


In [4]:
df_train

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,2.079442,True
1,1,2013-01-01,25,105574,0.693147,True
2,2,2013-01-01,25,105575,1.098612,True
3,3,2013-01-01,25,108079,0.693147,True
4,4,2013-01-01,25,108701,0.693147,True
...,...,...,...,...,...,...
125497035,125497035,2017-08-15,54,2089339,1.609438,False
125497036,125497036,2017-08-15,54,2106464,0.693147,True
125497037,125497037,2017-08-15,54,2110456,5.262690,False
125497038,125497038,2017-08-15,54,2113914,5.293305,True


In [5]:
df_test

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False
...,...,...,...,...,...
3370459,128867499,2017-08-31,54,2132163,False
3370460,128867500,2017-08-31,54,2132318,False
3370461,128867501,2017-08-31,54,2132945,False
3370462,128867502,2017-08-31,54,2132957,False


In [6]:
items

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1
...,...,...,...,...
4095,2132318,GROCERY I,1002,0
4096,2132945,GROCERY I,1026,0
4097,2132957,GROCERY I,1068,0
4098,2134058,BEVERAGES,1124,0


In [7]:
stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [8]:
# 数据集包含从 2013 年到 2017 年的数据，
# 时间跨度非常大，四年的发展过程中会产 生很多的不确定性。
# 在利用太久远的数据对未来进行预测时会产生一定的噪声，并 且会存在分布上的差异，
# 这一点在 11.2 节也可以发现。另外出于对性能的考虑，最 终仅使用 2017 年的数据作为训练集。执行下述代码过滤 2017 年之前的数据：
df_2017 = df_train.loc[df_train.date>='2017-01-01']
del df_train

In [9]:
df_2017

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
101688779,101688779,2017-01-01,25,99197,0.693147,False
101688780,101688780,2017-01-01,25,103665,2.079442,False
101688781,101688781,2017-01-01,25,105574,0.693147,False
101688782,101688782,2017-01-01,25,105857,1.609438,False
101688783,101688783,2017-01-01,25,106716,1.098612,False
...,...,...,...,...,...,...
125497035,125497035,2017-08-15,54,2089339,1.609438,False
125497036,125497036,2017-08-15,54,2106464,0.693147,True
125497037,125497037,2017-08-15,54,2110456,5.262690,False
125497038,125497038,2017-08-15,54,2113914,5.293305,True


In [ ]:
# 接下来进行基本的数据格式转换，并最终以店铺、商品和时间为索引，构造是否促 销的数据表，
# 以便进行与促销或者未促销相关的统计，这样的构造方式有利于之后 的特征提取。
promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
promo_2017 = promo_2017.astype('int')

C:\Users\yx140\AppData\Local\Temp\ipykernel_24996\1480650265.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
C:\Users\yx140\AppData\Local\Temp\ipykernel_24996\1480650265.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)


In [ ]:
# 是否促销表
promo_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995              0           0           0           0   
          99197              0           0           0           0   
          103520             0           0           0           0   
          103665             0           0           0           0   
          105574             0           0           1           0   
...                        ...         ...         ...         ...   
54        2109909            0           0           0           0   
          2110456            0           0           0           0   
          2113343            0           0           0           0   
          2113914            0           0           0           0   
          2116416            0           0           0           0   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995              0           0           0           0   
          99197              0           0           0           0   
          103520             0           0           0           0   
          103665             0           0           0           0   
          105574             0           1           0           0   
...                        ...         ...         ...         ...   
54        2109909            0           0           0           0   
          2110456            0           0           0           0   
          2113343            0           0           0           0   
          2113914            0           0           0           0   
          2116416            0           0           0           0   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995              0           0  ...           0           0   
          99197              0           0  ...           0           0   
          103520             0           0  ...           0           0   
          103665             0           0  ...           0           0   
          105574             0           0  ...           0           0   
...                        ...         ...  ...         ...         ...   
54        2109909            0           0  ...           0           0   
          2110456            0           0  ...           0           0   
          2113343            0           0  ...           0           0   
          2113914            0           0  ...           1           1   
          2116416            0           0  ...           0           0   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995              0           0           0           0   
          99197              0           0           0           0   
          103520             0           0           0           0   
          103665             0           0           0           0   
          105574             0           0           0           0   
...                        ...         ...         ...         ...   
54        2109909            0           0           0           0   
          2110456            0           0           0           1   
          2113343            0           0           0           0   
          2113914            1           1           1           1   
          2116416            0           0           0           0   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995              0           0           0           0  
          99197              0           0           0           0  
          10352

In [11]:
df_2017 = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0) 
df_2017.columns = df_2017.columns.get_level_values(1)

In [ ]:
# 单元销量表
df_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   
...                        ...         ...         ...         ...   
54        2109909          0.0    0.000000    0.000000    0.000000   
          2110456          0.0    0.000000    0.000000    0.000000   
          2113343          0.0    0.000000    0.000000    0.000000   
          2113914          0.0    0.000000    0.000000    0.000000   
          2116416          0.0    0.000000    0.000000    0.000000   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   
...                        ...         ...         ...         ...   
54        2109909     0.000000    0.000000    0.000000    0.000000   
          2110456     0.000000    0.000000    0.000000    0.000000   
          2113343     0.000000    0.000000    0.000000    0.000000   
          2113914     0.000000    0.000000    0.000000    0.000000   
          2116416     0.000000    0.000000    0.000000    0.000000   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    1.098612    1.098612   
          99197       0.000000    0.693147  ...    0.000000    1.098612   
          103520      0.693147    0.693147  ...    0.000000    0.000000   
          103665      0.000000    2.079442  ...    0.693147    1.098612   
          105574      1.386294    2.302585  ...    0.000000    1.791759   
...                        ...         ...  ...         ...         ...   
54        2109909     0.000000    0.000000  ...    0.693147    0.693147   
          2110456     0.000000    0.000000  ...    7.203406    6.481577   
          2113343     0.000000    0.000000  ...    1.098612    0.000000   
          2113914     0.000000    0.000000  ...    2.890372    0.000000   
          2116416     0.000000    0.000000  ...    0.693147    0.693147   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.693147    0.000000   
          99197       0.000000    1.098612    0.000000    0.000000   
          103520      1.386294    0.000000    1.386294    0.693147   
          103665      0.000000    2.079442    2.302585    1.098612   
          105574      2.079442    1.945910    2.397895    1.791759   
...                        ...         ...         ...         ...   
54        2109909     0.000000    1.098612    0.693147    0.000000   
          2110456     6.586172    3.218876    0.000000    0.000000   
          2113343     0.000000    0.000000    0.693147    0.000000   
          2113914     2.397895    2.397895    1.609438    0.000000   
          2116416     0.000000    0.000000    0.000000    0.693147   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995       0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000  
          10352

In [16]:
# 历史平移特征和窗口统计特征是时间序列预测问题的核心特征，
# 这里仅简单地使用 历史平移特征（一个单位）和不同窗口大小的窗口统计特征作为基础特征。
# 下面实 现的是提取特征的通用代码：

def get_date_range(df, dt, forward_steps, periods, freq='D'): 
    return df[pd.date_range(start=dt-timedelta(days=forward_steps), periods=periods, freq=freq)]



In [17]:
def prepare_dataset(t2017, is_train=True): 
    X = pd.DataFrame({ # 历史平移特征，前1、2、3 天的销量 
        "day_1_hist": get_date_range(df_2017, t2017, 1, 1).values.ravel(), 
        "day_2_hist": get_date_range(df_2017, t2017, 2, 1).values.ravel(), 
        "day_3_hist": get_date_range(df_2017, t2017, 3, 1).values.ravel(), })
    for i in [7, 14, 21, 30]: 
        # 窗口统计特征，销量diff/mean/meidan/max/min/std 
        X['diff_{}_day_mean'.format(i)] = get_date_range(df_2017, t2017, i, i).diff(axis=1).mean(axis=1).values 
        X['mean_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).mean(axis=1).values 
        X['median_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).mean(axis=1).values 
        X['max_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).max(axis=1).values
        X['min_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).min(axis=1).values 
        X['std_{}_day'.format(i)] = get_date_range(df_2017, t2017, i, i).min(axis=1).values
    for i in range(7): 
        # 前4、10 周每周的平均销量 
        X['mean_4_dow{}_2017'.format(i)] = get_date_range(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values 
        X['mean_10_dow{}_2017'.format(i)] = get_date_range(df_2017, t2017, 70-i, 10, freq='7D').mean(axis=1).values 
    for i in range(16): 
        # 未来16 天是否为促销日
        X["promo_{}".format(i)] = promo_2017[str(t2017 + timedelta(days=i))].values.astype(np.uint8) 
    if is_train: 
        y = df_2017[pd.date_range(t2017, periods=16)].values 
        return X, y 
    return X

In [18]:
# 以7 月5 日后的第16 天作为最后一个训练集窗口，向前依次递推14 周得到14 个训练窗口的训练数据 
from tqdm import tqdm
X_l, y_l = [], [] 
t2017 = date(2017, 7, 5) 
n_range = 14 
for i in tqdm(range(n_range)): 
    delta = timedelta(days=7 * i) 
    X_tmp, y_tmp = prepare_dataset(t2017 - delta) 
    X_l.append(X_tmp) 
    y_l.append(y_tmp) 
X_train = pd.concat(X_l, axis=0) 
y_train = np.concatenate(y_l, axis=0) 
del X_l, y_l 

# 验证集取7 月26 日到8 月10 日的数据 
X_val, y_val = prepare_dataset(date(2017, 7, 26)) 

# 测试集取8 月16 日到8 月31 日的数据
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

100%|██████████| 14/14 [00:16<00:00,  1.18s/it]


In [24]:
X_train

,day_1_hist,day_2_hist,day_3_hist,diff_7_day_mean,mean_7_day,median_7_day,max_7_day,min_7_day,std_7_day,diff_14_day_mean,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.000000,0.000000,0.000000,0.000000e+00,0.099021,0.099021,0.693147,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000,0.000000,-2.310491e-01,0.454008,0.454008,1.386294,0.0,0.0,-0.053319,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.693147,0.000000,-2.310491e-01,0.709973,0.709973,1.386294,0.0,0.0,-0.053319,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.000000,0.693147,1.850372e-17,0.930037,0.930037,2.484907,0.0,0.0,-0.053319,...,0,0,0,0,0,0,0,0,0,0
4,2.197225,2.397895,0.000000,6.757752e-02,1.523364,1.523364,2.397895,0.0,0.0,-0.057478,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167510,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
167511,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
167512,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
167513,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X_val
y_val

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.69314718],
       [0.        , 0.        , 0.69314718, ..., 0.        , 1.09861229,
        0.        ],
       [0.69314718, 1.09861229, 1.09861229, ..., 1.38629436, 0.        ,
        1.38629436],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.69314718],
       [1.94591015, 1.38629436, 1.09861229, ..., 2.39789527, 2.39789527,
        1.60943791],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [23]:
from lightgbm import log_evaluation
params = {'num_leaves': 2 ** 5 - 1, 'objective': 'regression_l2', 'max_depth': 8, 'min_data_in_leaf': 50,
          'learning_rate': 0.05, 'feature_fraction': 0.75, 'bagging_fraction': 0.75, 'bagging_freq': 1, 'metric': 'l2',
          'num_threads': 4}
MAX_ROUNDS = 500
val_pred = []
test_pred = []
callbacks = [log_evaluation(period=100)]
for i in range(16): 
    print("====== Step %d ======" % (i + 1))
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain)
    bst = lgb.train(params, dtrain, num_boost_round=MAX_ROUNDS, valid_sets=[dtrain, dval], callbacks=callbacks)
    val_pred.append(bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

====== Step 1 ======
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10487
[LightGBM] [Info] Number of data points in the train set: 2345210, number of used features: 57
[LightGBM] [Info] Start training from score 1.018067
[100]	training's l2: 0.314394	valid_1's l2: 0.303008
[200]	training's l2: 0.310553	valid_1's l2: 0.300609
[300]	training's l2: 0.30874	valid_1's l2: 0.299649
[400]	training's l2: 0.307396	valid_1's l2: 0.299107
[500]	training's l2: 0.306384	valid_1's l2: 0.298779
====== Step 2 ======
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121650 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10487
[LightGBM] [Info] Number